<a href="https://colab.research.google.com/github/kimberly-kao/CS4650-sPacyPractice/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!pip install newsapi-python



In [2]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 4.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
from newsapi import NewsApiClient
import spacy

API_KEY = "9d23ebd5b162489f9eddc9f92ad4828b"
nlp_eng = spacy.load('en')
newsapi = NewsApiClient (API_KEY)

In [4]:
pos_tag = ["VERB", "NOUN", "PROPN"]

def get_keywords_eng(doc):
  result = []
  for token in doc:
    if (token.text in nlp_eng.Defaults.stop_words or token.is_punct):
      continue
    if (token.pos_ in pos_tag):
      result.append(token.text)
  return result

In [9]:
import pickle
import pandas as pd
from collections import Counter

articles = []

for x in range(5):
  x = x+1
  articles.append(newsapi.get_everything(q='coronavirus', language='en', 
                              from_param='2022-02-25', to='2022-03-22', 
                              sort_by='relevancy', page=x))

filename = 'articlesCOVID.pckl'
pickle.dump(articles, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
filepath = '/articlesCOVID.pckl'
pickle.dump(loaded_model, open(filepath, 'wb'))

dados = []

for i, article in enumerate(articles):
    for x in article['articles']:
        title = x['title']
        description = x['description']
        content = x['content']
        date = x['publishedAt']
        dados.append({'title':title, 'date':date, 'desc':description, 'content':content})

df = pd.DataFrame(dados)
df = df.dropna()
df.head()

results = []


for content in df.content.values:
    content = nlp_eng(content)
    results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
df['keywords'] = results

